In [2]:
from pathlib import Path

try:
    import yaml
except ImportError as exc:
    raise RuntimeError("PyYAML is required to load the dataset config. Install it with 'pip install pyyaml'.") from exc

from ultralytics import YOLO

# Build a repo-relative path so the YAML resolves regardless of the working directory
if "__file__" in globals():
    repo_root = Path(__file__).resolve().parent.parent
else:
    repo_root = Path.cwd()
    if not (repo_root / "datasets").exists() and (repo_root.parent / "datasets").exists():
        repo_root = repo_root.parent

data_yaml = repo_root / "datasets" / "vg" / "VG150" / "YOLO_anno" / "train.yaml"
if not data_yaml.exists():
    raise FileNotFoundError(f"Dataset YAML not found at {data_yaml}")

with data_yaml.open("r", encoding="utf-8") as f:
    raw_config = yaml.safe_load(f)

dataset_root = data_yaml.parent
resolved_config = {
    "path": str(dataset_root),
    "train": str(dataset_root / "train"),
    "val": str(dataset_root / "val"),
    "test": str(dataset_root / "test"),
}

for split_name in ("train", "val", "test"):
    split_dir = dataset_root / split_name
    if not split_dir.exists():
        raise FileNotFoundError(f"Expected dataset split folder missing: {split_dir}")

names = raw_config.get("names")
if names:
    resolved_config["names"] = names
    resolved_config["nc"] = len(names)


In [3]:
resolved_config

{'path': 'e:\\Work\\CT UAV\\Projects\\sgg-feedback\\SGG-Benchmark\\datasets\\vg\\VG150\\YOLO_anno',
 'train': 'e:\\Work\\CT UAV\\Projects\\sgg-feedback\\SGG-Benchmark\\datasets\\vg\\VG150\\YOLO_anno\\train',
 'val': 'e:\\Work\\CT UAV\\Projects\\sgg-feedback\\SGG-Benchmark\\datasets\\vg\\VG150\\YOLO_anno\\val',
 'test': 'e:\\Work\\CT UAV\\Projects\\sgg-feedback\\SGG-Benchmark\\datasets\\vg\\VG150\\YOLO_anno\\test',
 'names': ['bag',
  'basket',
  'bin',
  'blind',
  'book',
  'bookshelf',
  'bottle',
  'bowl',
  'box',
  'button',
  'cabinet',
  'camera',
  'can',
  'carpet',
  'ceiling',
  'chair',
  'clock',
  'computer',
  'couch',
  'counter',
  'cup',
  'curtain',
  'desk',
  'dishwasher',
  'door',
  'drawer',
  'eyeglasses',
  'faucet',
  'floor',
  'flower',
  'food',
  'frame',
  'fruit',
  'glass',
  'hand',
  'handle',
  'head',
  'jacket',
  'key',
  'keyboard',
  'knife',
  'knob',
  'lamp',
  'laptop',
  'leg',
  'lid',
  'light',
  'luggage',
  'magazine',
  'microwave',


In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("../backbone/yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="../datasets/VG/VG150/YOLO_anno/train.yaml", epochs=1, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.217 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.193  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/VG/VG150/YOLO_anno/train.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../backbone/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3